In [1]:
import numpy as np
import sympy as sp
import matplotlib as mpl

Source: W. Dahmen, A. Reusken "Numerik für Ingenieure und Naturwissenshaftler" 2 Auflage

# The condition number of a problem 

The condition number of a problem measures how the sensitive the output of a problem (or function) is to an error in the input data. It can be defined relative, as the maximum component in the case of a function defined in multiple dimensions ($f: \mathbb{R}^n \rightarrow \mathbb{R}$, as the largest component of the gradient, relative to what is called the error amplification factor:
$$
\kappa_{rel}(x) = \kappa_{rel}^{\infty}(x) = \max_{j=1\dots n} \left| \frac{\partial f(x)}{\partial x_j} \frac{x_j}{f(x)} \right|
$$

## Example (2.12, page 23)

Given the function $f: \mathbb{R} \rightarrow \mathbb{R}$, $f(x) = \exp{(3x^2)}$. The relative condition number is given by:

\begin{align*}
\kappa_{rel}(x) &= \left| f'(x) \frac{x}{f(x)} \right| \\
&= \left| 6x \exp{(3x^2)} \frac{x}{\exp{(3x^2)}} \right| \\
&= 6x^2
\end{align*}


From this it is evident that the relative condition number would be small for small input magnitude but grows fast.

In [2]:
def f(x):
    return np.exp(3*x**2) 

def rel_err(val, valp):
    return np.abs((val - valp)/val)

In [3]:
x1 = 0.1
x1p = 0.10001
ie1 = rel_err(x1,x1p)
ie1

9.999999999996123e-05

In [4]:
y1 = f(x1)
y1p = f(x1p)
oe1 = rel_err(y1, y1p)
oe1

6.000318001768653e-06

While arround a bigger $x$, for the same input error, the output error will be much bigger

In [5]:
x2 = 4
x2p = 4.0004
ie2 = rel_err(x2,x2p)
ie2

9.999999999998899e-05

In [6]:
y2 = f(x2)
y2p = f(x2p)
oe2 = rel_err(y2, y2p)
oe2

0.009646712440876656

In [7]:
q = oe2 / oe1
q

1607.7001982283593

To summarize, I understand what is happening here in the following way. The analysis starts with the taylor series of the function around $x$. For the function $f: \mathbb{R}^n \rightarrow \mathbb{R}$

$$
f(\tilde{x}) = f(x) + (\nabla f(x) )^T (\tilde{x} - x) + \mathcal{O}(\Vert \tilde{x} - x \Vert^2), \quad (\tilde{x} \rightarrow x).
$$

When $\Vert \tilde{x} - x \Vert^2$ is small, $\tilde{x}$ and $x$ are close to each other and one can ignore the second order term
$$
f(\tilde{x}) \dot{=} f(x) + (\nabla f(x))^T (\tilde{x} - x)
$$

This this first order approximation, can be rewritten to reveal input output relationship, as above, between the relative input and output

$$
f(\tilde{x}) - f(x) \dot{=} \sum_{j=1}^{n} \frac{\partial f}{\partial x_j} (\tilde{x}_j - x_j)
$$


Dividing both sides by $f(x)$ and multipliying each of the sum terms by $1 = \frac{x_j}{x_j}$ for $j=1\dots n$ 

$$
\frac{f(\tilde{x}) - f(x)}{f(x)} \dot{=} \sum_{j=1}^{n} \frac{\partial f}{\partial x_j} \cdot \frac{x_j}{f(x)} \cdot \frac{\tilde{x}_j - x_j}{x_j}
$$

define the amplification factors

$$
\phi_j(x) := \frac{\partial f}{\partial x_j} \cdot \frac{x_j}{f(x)}
$$

Results in the formula for the relative output error as the sum of each of the components of the amplified input relative error. 

$$
\frac{f(\tilde{x}) - f(x)}{f(x)} \dot{=} \sum_{j=1}^n \phi_j(x) \cdot \frac{\tilde{x}_j - x_j}{x_j}
$$

which magnitude can be bounded above by defining the relative condition numerber as the maximum absolute value of the amplifiying factors, as done above, 

$$
\kappa_{rel}(x) = \kappa_{rel}^{\infty}(x) = \max_{j} \left| \frac{\partial f}{\partial x_j} \cdot \frac{x_j}{f(x)} \right|
$$

$$
\left| \frac{f(\tilde{x} - f(x)}{f(x)} \right| \dot{\leq} \kappa_{rel}(x) \sum_{j=1}^n \left| \frac{\tilde{x}_j - x_j}{x_j} \right|
$$

Arithmetic operations: addition, multiplication and division are well conditioned, only substraction can be arbitrarily ill-conditioned. Consider the addition example:

## Addition (example 2.14)

In [21]:
x1, x2 = sp.symbols('x1, x2')
x = sp.Matrix([[x1],[x2]])
f = (sp.Matrix([[1, 1]])*x)[0,0]
f

x1 + x2

In [23]:
dfdx1 = sp.diff(f,x1)
dfdx1

1

In [24]:
dfdx2 = sp.diff(f,x2)
dfdx2

1

In [25]:
phi1 = dfdx1 * x1 / f
phi1

x1/(x1 + x2)

In [27]:
phi2 = dfdx2 * x2 / f
phi2

x2/(x1 + x2)

thus the output relative error is bounded for the addition, since

\begin{align*}
\left| \frac{f(\tilde{x}) - f(x)}{f(x)} \right| &= \left| \frac{(\tilde{x}_1 + \tilde{x}_2) - (x_1 + x_2)}{x_1 + x_2} \right| \\
&\leq \kappa_{rel} \left( \left| \frac{\tilde{x}_1 - x_1}{x_1}\right| + \left| \frac{\tilde{x}_2 - x_2}{x_2} \right| \right) \\
&\leq \kappa_{rel} 2 \epsilon
\end{align*}

where: $\left| \frac{\tilde{x}_1 - x_1}{x_1} \right|,\, \left| \frac{\tilde{x}_2 - x_2}{x_2} \right| \leq \epsilon$

## Substraction!

Interestingly, a substraction is a sum with $x_1$ and $x_2$ having different sings. It is problematic when the magnitudes of $x_1$ and $x_2$ are similar and the result approaches 0. In this case holds,

$$
\left| x_1 + x_2 \right| << |x_i|
$$

in this case the factors $\phi_j(x)$, $j=1,2$ can be arbitrarily large, and $\kappa_{rel}(x) >> 1$ cannot be bound by a constant.